In [2]:
import torch
import torch.nn as nn
import numpy as np

In [12]:
import seaborn as sns
import torch

# ডেটাসেট লোড
iris = sns.load_dataset('iris')

# ইনপুট ফিচারগুলোকে টেনসরে রূপান্তর
data = torch.tensor(iris[iris.columns[0:4]].values).float()  # 🔸 .float() ব্যবহার করো dtype মিলাতে

# লেবেল ইনিশিয়ালাইজ
labels = torch.zeros(len(data), dtype=torch.long)

# 🔸 spelling mistake ফিক্স করা হয়েছে: 'verginica' → 'virginica'
labels[iris.species == 'versicolor'] = 1
labels[iris.species == 'virginica'] = 2


### Separate data into train and test

In [13]:
propTaining=.8
nTraining=int(len(labels)*propTaining)

##initialize a boolean vector to select data and labels
traintestBool=np.zeros(len(labels),dtype=bool)

#is this the correct way to select sample?
traintestBool[range(nTraining)]=True

## this is better but why?
traintestBool

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False, False, False, False, False,
       False, False,

In [14]:
#test whether it's balance
print('Average of full data:')
print(torch.mean(labels.float()))
print(' ')

print('Average of training data:')
print(torch.mean(labels[traintestBool].float()))
print(' ')

print('Average of test data:')
print(torch.mean(labels[~traintestBool].float()))

Average of full data:
tensor(1.)
 
Average of training data:
tensor(0.7500)
 
Average of test data:
tensor(2.)


In [15]:
##create the ANN model
ANNiris=nn.Sequential(
    nn.Linear(4,64),
    nn.ReLU(),
    nn.Linear(64,64),
    nn.ReLU(),
    nn.Linear(64,3),
)

##loss function
lossfun=nn.CrossEntropyLoss()

#optimizer
optimizer=torch.optim.SGD(ANNiris.parameters(),lr=.01)




In [16]:
#entire datasets
print(data.shape)

#training set
print(data[traintestBool,:].shape)

#test set
print(data[~traintestBool,:].shape)

torch.Size([150, 4])
torch.Size([120, 4])
torch.Size([30, 4])


### Train and test the model

In [17]:
# হাইপার-প্যারামিটার
numepochs = 1000
losses = torch.zeros(numepochs)
ongoingAcc = []

# মডেল ট্রেন করা শুরু
for epochi in range(numepochs):

    # 🔹 Forward pass
    yHat = ANNiris(data[traintestBool, :])

    # 🔹 Accuracy হিসাব করা
    acc = 100 * torch.mean((torch.argmax(yHat, axis=1) == labels[traintestBool]).float())
    ongoingAcc.append(acc.item())  # item() দিয়ে tensor থেকে value আনা হয়েছে

    # 🔹 Loss হিসাব করা
    loss = lossfun(yHat, labels[traintestBool])
    losses[epochi] = loss.item()  # tensor value সংরক্ষণ

    # 🔹 Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [18]:
#compute train and tets accuracies

predictions=ANNiris(data[traintestBool,:])
trainacc=100*torch.mean((torch.argmax(predictions,axis=1)==labels[traintestBool]).float())

predictions=ANNiris(data[~traintestBool,:])
tetsacc=100*torch.mean((torch.argmax(predictions,axis=1)==labels[~traintestBool]).float())


In [19]:
print('Final Train Accuracy: %g%%' %trainacc)
print('Final Test Accuracy: %g%%' %tetsacc)

Final Train Accuracy: 98.3333%
Final Test Accuracy: 73.3333%


### Additional explorations